In [1]:
import pygame
import numpy as np
import sys
import random

# Pygame setup
pygame.init()
screen_width, screen_height = 1200, 900
screen = pygame.display.set_mode((screen_width, screen_height))
clock = pygame.time.Clock()

# Boid and Cloid settings
num_boids = 1
boid_size = 3  # Used for the triangle size
boid_view_radius = 270
boid_max_speed = 5
boid_min_speed = 2  # Minimum speed for boids
boid_max_force = 0.1

num_cloids = 1
cloid_size = 5  # Used for the triangle size
cloid_view_radius = 90
cloid_max_speed = 5
cloid_min_speed = 2  # Minimum speed for cloids
cloid_max_force = 0.1

# Colors
background_color = pygame.Color('darkslategray')
boid_color = pygame.Color('white')
cloid_color = pygame.Color('orange')

def wrap_around(position):
    x, y = position
    return np.array([x % screen_width, y % screen_height], dtype=float)
    
def angle_between(v1, v2):
    """Calculate the angle in radians between vectors 'v1' and 'v2'"""
    unit_v1 = v1 / np.linalg.norm(v1)
    unit_v2 = v2 / np.linalg.norm(v2)
    dot_product = np.dot(unit_v1, unit_v2)
    angle = np.arccos(np.clip(dot_product, -1.0, 1.0))
    return angle


    
class Boid:
    def __init__(self, x, y):
        self.position = np.array([x, y], dtype=float)
        self.velocity = (np.random.rand(2) * 2 - 1) * boid_max_speed
        self.acceleration = np.zeros(2, dtype=float)

    def update(self):
        self.velocity += self.acceleration
        speed = np.linalg.norm(self.velocity)
        if speed > boid_max_speed:
            self.velocity = (self.velocity / speed) * boid_max_speed
        elif speed < boid_min_speed:
            self.velocity = (self.velocity / speed) * boid_min_speed
        self.position += self.velocity
        self.acceleration *= 0
        self.position = wrap_around(self.position)

   
    def can_see(self, other_boid):
        """Check if this boid can 'see' the other boid within a 270-degree view."""
        forward_direction = self.velocity / np.linalg.norm(self.velocity)
        to_other_boid = other_boid.position - self.position
        if np.linalg.norm(to_other_boid) == 0:
            return False  # Ignore self
        to_other_boid /= np.linalg.norm(to_other_boid)
        
        view_angle = angle_between(forward_direction, to_other_boid)
        return view_angle < np.radians(135)  # 270-degree view divided by 2 for either side

    def draw(self, screen):
        # Calculate the angle of the velocity
        angle = np.arctan2(self.velocity[1], self.velocity[0])
        # Define the triangle points based on the boid's position and angle
        points = [self.position + np.array([np.cos(angle), np.sin(angle)]) * boid_size,
                  self.position + np.array([np.cos(angle + 5*np.pi/6), np.sin(angle + 5*np.pi/6)]) * boid_size,
                  self.position + np.array([np.cos(angle - 5*np.pi/6), np.sin(angle - 5*np.pi/6)]) * boid_size]
        pygame.draw.polygon(screen, boid_color, points)

    def apply_behaviors(self, boids):
        separation_force = self.separation(boids) * 1.5
        alignment_force = self.alignment(boids) * 1.0
        cohesion_force = self.cohesion(boids) * 1.0
        
        self.acceleration += separation_force + alignment_force + cohesion_force

    def separation(self, boids):
        steering = np.zeros(2)
        total = 0
        for boid in boids:
            distance = np.linalg.norm(self.position - boid.position)
            if self != boid and self.can_see(boid) and distance < boid_view_radius / 2:
                diff = self.position - boid.position
                diff /= distance  # Weight by distance
                steering += diff
                total += 1
        if total > 0:
            steering /= total
        steering = np.clip(steering, -boid_max_force, boid_max_force)
        return steering

    def alignment(self, boids):
        avg_velocity = np.zeros(2)
        total = 0
        for boid in boids:
            if self != boid and self.can_see(boid) and np.linalg.norm(self.position - boid.position) < boid_view_radius:
                avg_velocity += boid.velocity
                total += 1
        if total > 0:
            avg_velocity /= total
        return np.clip(avg_velocity - self.velocity, -boid_max_force, boid_max_force)

    def cohesion(self, boids):
        center_of_mass = np.zeros(2)
        total = 0
        for boid in boids:
            if self != boid and self.can_see(boid) and np.linalg.norm(self.position - boid.position) < boid_view_radius:
                center_of_mass += boid.position
                total += 1
        if total > 0:
            center_of_mass /= total
            return np.clip((center_of_mass - self.position) / 100, -boid_max_force, boid_max_force)
        return np.zeros(2)





class Cloid:
    def __init__(self, x, y):
        self.position = np.array([x, y], dtype=float)
        self.velocity = (np.random.rand(2) * 2 - 1) * cloid_max_speed
        self.acceleration = np.zeros(2, dtype=float)

    def update(self):
        self.velocity += self.acceleration
        speed = np.linalg.norm(self.velocity)
        if speed > cloid_max_speed:
            self.velocity = (self.velocity / speed) * cloid_max_speed
        elif speed < cloid_min_speed:
            self.velocity = (self.velocity / speed) * cloid_min_speed
        self.position += self.velocity
        self.acceleration *= 0
        self.position = wrap_around(self.position)

    
    def can_see(self, other_cloid):
        """Check if this cloid can 'see' the other cloid within a 270-degree view."""
        forward_direction = self.velocity / np.linalg.norm(self.velocity)
        to_other_cloid = other_cloid.position - self.position
        if np.linalg.norm(to_other_cloid) == 0:
            return False  # Ignore self
        to_other_cloid /= np.linalg.norm(to_other_cloid)
        
        view_angle = angle_between(forward_direction, to_other_cloid)
        return view_angle < np.radians(135)  # 270-degree view divided by 2 for either side

    def draw(self, screen):
        # Calculate the angle of the velocity
        angle = np.arctan2(self.velocity[1], self.velocity[0])
        # Define the triangle points based on the cloid's position and angle
        points = [self.position + np.array([np.cos(angle), np.sin(angle)]) * cloid_size,
                  self.position + np.array([np.cos(angle + 5*np.pi/6), np.sin(angle + 5*np.pi/6)]) * cloid_size,
                  self.position + np.array([np.cos(angle - 5*np.pi/6), np.sin(angle - 5*np.pi/6)]) * cloid_size]
        pygame.draw.polygon(screen, cloid_color, points)

    def apply_behaviors(self, cloids):
        separation_force = self.separation(cloids) * 1.5
        alignment_force = self.alignment(cloids) * 1.0
        cohesion_force = self.cohesion(cloids) * 1.0
        
        self.acceleration += separation_force + alignment_force + cohesion_force

    def separation(self, cloids):
        steering = np.zeros(2)
        total = 0
        for cloid in cloids:
            distance = np.linalg.norm(self.position - cloid.position)
            if self != cloid and self.can_see(cloid) and distance < cloid_view_radius / 2:
                diff = self.position - cloid.position
                diff /= distance  # Weight by distance
                steering += diff
                total += 1
        if total > 0:
            steering /= total
        steering = np.clip(steering, -cloid_max_force, cloid_max_force)
        return steering

    def alignment(self, cloids):
        avg_velocity = np.zeros(2)
        total = 0
        for cloid in cloids:
            if self != cloid and self.can_see(cloid) and np.linalg.norm(self.position - cloid.position) < cloid_view_radius:
                avg_velocity += cloid.velocity
                total += 1
        if total > 0:
            avg_velocity /= total
        return np.clip(avg_velocity - self.velocity, -cloid_max_force, cloid_max_force)

    def cohesion(self, cloids):
        center_of_mass = np.zeros(2)
        total = 0
        for cloid in cloids:
            if self != cloid and self.can_see(cloid) and np.linalg.norm(self.position - cloid.position) < cloid_view_radius:
                center_of_mass += cloid.position
                total += 1
        if total > 0:
            center_of_mass /= total
            return np.clip((center_of_mass - self.position) / 100, -cloid_max_force, cloid_max_force)
        return np.zeros(2)



print("Screen width:", screen_width)
print("Screen height:", screen_height)
cloids = [Cloid(np.random.randint(0, screen_width), np.random.randint(0, screen_height)) for _ in range(num_cloids)]



# Inside the list comprehension for initializing boids
boids = [Boid(random.randint(0, screen_width-1), random.randint(0, screen_height-1)) for _ in range(num_boids)]
cloids = [Cloid(random.randint(0, screen_width-1), random.randint(0, screen_height-1)) for _ in range(num_cloids)]

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            keys = pygame.key.get_pressed()
            if keys[pygame.K_LSHIFT] or keys[pygame.K_RSHIFT]:  # Check if either Shift key is pressed
                mouse_x, mouse_y = pygame.mouse.get_pos()
                new_cloid = Cloid(mouse_x, mouse_y)
                cloids.append(new_cloid)
            else:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                new_boid = Boid(mouse_x, mouse_y)
                boids.append(new_boid)

    # Ensure the screen is filled every frame before drawing boids and cloids
    screen.fill(background_color)
    for boid in boids:
        boid.update()
        boid.draw(screen)
    for cloid in cloids:
        cloid.update()
        cloid.draw(screen)
    pygame.display.flip()
    clock.tick(60)

pygame.quit()

pygame 2.5.2 (SDL 2.28.2, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
Screen width: 1200
Screen height: 900
